In [15]:
import cv2
import numpy as np

In [32]:
# Load COCO class labels
class_labels = []
with open("coco.names", "rt") as f:
    class_labels = f.read().rstrip('\n').split('\n')

# Load pretrained YOLO model
# Load the pre-trained Faster R-CNN model and configuration
model_path = 'frozen_inference_graph.pb'
config_path = 'faster_rcnn_inception_v2_coco_2018_01_28.pbtxt'
net = cv2.dnn.readNetFromTensorflow(model_path, config_path)

In [45]:
image = cv2.imread("Bus_rides2.jpg")
height, width = image.shape[:2]

# Create a 4D blob from the image (resize, normalize, swap channels)
blob = cv2.dnn.blobFromImage(image, scalefactor=1/255.0, size=(416, 416), swapRB=True, crop=False)

# Set the input to the network
net.setInput(blob)

In [46]:
# Get output layer names
out_layer_names = net.getUnconnectedOutLayersNames()

# Forward pass to get output
output = net.forward(out_layer_names)

In [47]:
# Perform object detection
detections = net.forward()

# Loop through the detections
for detection in detections[0, 0, :, :]:
    score = float(detection[2])
    class_id = int(detection[1])
    
    if score > 0.1 and class_id < len(class_labels):  # Filter detections with confidence greater than 50%
        height, width, _ = image.shape
        left = int(detection[3] * width)
        top = int(detection[4] * height)
        right = int(detection[5] * width)
        bottom = int(detection[6] * height)

        # Draw bounding box and label on the image
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f'{class_labels[class_id]}: {score:.2f}'
        cv2.putText(image, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [48]:
# Display the result
cv2.imshow('Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()